<a href="https://colab.research.google.com/github/Alebeiruth/KNN_exerc/blob/main/TDE_01_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabalho 01 - Implementação de Modelo Supervisionado - TDE
# Utilizando KNN
# GRUPO AM 6

# Bibliotecas

In [ ]:
# modulos gerais
import numpy as np
import pandas as pd
import time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# modulos de vizualização
import seaborn as sns
import matplotlib.pyplot as plt

# modulos para download e obtenção dos dados, 2 datas sets selecionados Iris e Digits
from sklearn.datasets import load_iris
from sklearn.datasets import load_digits

# Funções

In [ ]:
# função da formula da distancia Euclidiana
def euclidiana_distancia(tamanho_a, tamanho_b):
    soma = 0
    for i in range(len(tamanho_a)):
        soma += (tamanho_a[i] - tamanho_b[i]) ** 2

    return soma ** 0.5

In [ ]:
# função que calcula as distancias entre elementos de teste e treino
def calcular_distancias(X_train, y_train, elemento_teste):
    distancias = []

    for i in range(len(X_train)):
        # Calcula a distância Euclidiana entre o elemento de teste e de treino
        distancia = euclidiana_distancia(X_train[i], elemento_teste)
        # Armazena o índice, a classe do elemento e a distância em uma tupla
        distancias.append((i, y_train[i], distancia))

    return distancias

In [ ]:
# função que ordena as disntancias encontradas
def ordenar_distancias(distancias):
    # Ordena a lista de tuplas pela distância (terceiro elemento da tupla ou indice [2])
    distancias_ordenadas = sorted(distancias, key=lambda x: x[2])
    return distancias_ordenadas

In [ ]:
# função para selecionar k vizinhos proximos
def selecionar_vizinhos(distancias_ordenadas, k):
    # Seleciona os k vizinhos
    vizinhos = distancias_ordenadas[:k]
    return vizinhos

# função que vota na classe predominante
def votar(vizinhos):
    contagem_classes = {}
    # Extrai as classes dos vizinhos
    for v in vizinhos:
        classe = v[1] # contem a classe do vizinho
        if classe in contagem_classes:
            contagem_classes[classe] += 1
        else:
            contagem_classes[classe] = 1

    # Retorna a classe mais comum entre os vizinhos (votação)
    classe_predominante = max(contagem_classes, key=contagem_classes.get)
    return classe_predominante


# 1 - Primeira base de dados - Digits

In [ ]:
# carregamento da base de dados e verificação do formato
X_digits, y_digits = load_digits(return_X_y=True)
print(X_digits.shape)
print(y_digits.shape)

(1797, 64)
(1797,)


In [ ]:
# visualização dos dados em DataFrame
x = pd.DataFrame(X_digits)
y = pd.DataFrame(y_digits)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,3.0,11.0,16.0,9.0,0.0
3,0.0,0.0,7.0,15.0,13.0,1.0,0.0,0.0,0.0,8.0,...,9.0,0.0,0.0,0.0,7.0,13.0,13.0,9.0,0.0,0.0
4,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,16.0,4.0,0.0,0.0


In [ ]:
# houldout/separação dos dados em treinamento e teste
X_train_digits, X_test_digits, y_train_digits, y_test_digits = train_test_split(X_digits, y_digits, test_size=0.3, random_state=42, stratify=y_digits)

In [ ]:
# verificação dos formatos
print("X_test_digits.: ", X_test_digits.shape)
print("y_test_digits.: ", y_test_digits.shape)
print("X_train_digits.: ", X_train_digits.shape)
print("y_train_digits.: ", y_train_digits.shape)

X_test_digits.:  (540, 64)
y_test_digits.:  (540,)
X_train_digits.:  (1257, 64)
y_train_digits.:  (1257,)


In [ ]:
# Função do KNN Manual somente para Acuracia
def knn_manual_acuracia(k, X_train, y_train, X_test, y_test):
    predicao_corretas = 0

    for i in range(len(X_test)):
        distancias_teste = calcular_distancias(X_train, y_train, X_test[i])
        distancias_ordenadas = ordenar_distancias(distancias_teste)
        vizinhos = selecionar_vizinhos(distancias_ordenadas, k)
        classe_predita = votar(vizinhos)

        # Check if the prediction is correct
        if classe_predita == y_test[i]:
            predicao_corretas += 1

    # Calcula Acurácia
    acuracia = predicao_corretas / len(X_test)
    return f"Acurácia do nosso experimento com K = {k}: {acuracia * 100:.2f}% ou {acuracia:.5f}"

# Função do KNN Manual somente para uso do classification_report
def knn_manual_rotulo(k, X_train, y_train, X_test, y_test):
    rotulos_preditos = []
    predicao_corretas = 0

    for i in range(len(X_test)):
        distancias_teste = calcular_distancias(X_train, y_train, X_test[i])
        distancias_ordenadas = ordenar_distancias(distancias_teste)
        vizinhos = selecionar_vizinhos(distancias_ordenadas, k)
        classe_predita = votar(vizinhos)
        rotulos_preditos.append(classe_predita)

        # Check if the prediction is correct
        if classe_predita == y_test[i]:
            predicao_corretas += 1

    # Calcula Acurácia
    acuracia = predicao_corretas / len(X_test)

    return rotulos_preditos, acuracia

## Utilização da função do algoritmo KNN da biblioteca Scikit-Learn
def knn_scikit_learn(k, X_train, y_train, X_test, y_test):
    knn = KNeighborsClassifier(n_neighbors=k)
    classif = knn.fit(X_train, y_train)
    score = classif.score(X_test, y_test)
    predicted = classif.predict(X_test)
    matrix = confusion_matrix(y_test, predicted)
    return f'Acurácia Scikit-Learn com K = {k}: {score* 100:.2f}% ou {score:.5f} \n Matrix de Confusão:\n{matrix}\n'


# Chamada das funções - Digits

In [ ]:
# Inicia o timer
tempo_inicial1 = time.time()

# Chamada com K = 3, com KNN manual e Sckiti-Learn, Matriz de COnfusão e Classification Report
print(knn_manual_acuracia(3, X_train_digits, y_train_digits, X_test_digits, y_test_digits))
print(knn_scikit_learn(3, X_train_digits, y_train_digits, X_test_digits, y_test_digits))
predicted_manual_knn, accuracy_manual_knn = knn_manual_rotulo(3, X_train_digits, y_train_digits, X_test_digits, y_test_digits)
print(classification_report(y_test_digits, predicted_manual_knn))

# Para o timer e calcula o tempo de execução
tempo_final1 = time.time()
tempo_total1 = tempo_final1 - tempo_inicial1
print(f"Tempo de execução com k = 3: {tempo_total1:.2f} segundos\n")

# Inicia o timer
tempo_inicial2 = time.time()

# Chamada com K = 13, com KNN manual e Sckiti-Learn, Matriz de COnfusão e Classification Report
print(knn_manual_acuracia(13, X_train_digits, y_train_digits, X_test_digits, y_test_digits))
print(knn_scikit_learn(13, X_train_digits, y_train_digits, X_test_digits, y_test_digits))
predicted_manual_knn, accuracy_manual_knn = knn_manual_rotulo(13, X_train_digits, y_train_digits, X_test_digits, y_test_digits)
print(classification_report(y_test_digits, predicted_manual_knn))

# Para o timer e calcula o tempo de execução
tempo_final2 = time.time()
tempo_total2 = tempo_final2 - tempo_inicial2
print(f"Tempo de execução com K = 13: {tempo_total2:.2f} segundos\n")

# Inicia o timer
tempo_inicial3 = time.time()

# Chamada com K = 31, com KNN manual e Sckiti-Learn, Matriz de COnfusão e Classification Report
print(knn_manual_acuracia(31, X_train_digits, y_train_digits, X_test_digits, y_test_digits))
print(knn_scikit_learn(31, X_train_digits, y_train_digits, X_test_digits, y_test_digits))
predicted_manual_knn, accuracy_manual_knn = knn_manual_rotulo(31, X_train_digits, y_train_digits, X_test_digits, y_test_digits)
print(classification_report(y_test_digits, predicted_manual_knn))

# Para o timer e calcula o tempo de execução
tempo_final3 = time.time()
tempo_total3 = tempo_final3 - tempo_inicial3
print(f"Tempo de execução com K = 31: {tempo_total3:.2f} segundos\n")

Acurácia do nosso experimento com K = 3: 98.70% ou 0.98704
Acurácia Scikit-Learn com K = 3: 98.70% ou 0.98704 
 Matrix de Confusão:
[[54  0  0  0  0  0  0  0  0  0]
 [ 0 55  0  0  0  0  0  0  0  0]
 [ 0  0 53  0  0  0  0  0  0  0]
 [ 0  0  0 55  0  0  0  0  0  0]
 [ 0  0  0  0 54  0  0  0  0  0]
 [ 0  0  0  0  0 54  0  0  0  1]
 [ 0  0  0  0  0  0 54  0  0  0]
 [ 0  0  0  0  0  0  0 54  0  0]
 [ 0  3  0  0  0  0  1  0 48  0]
 [ 0  0  0  0  1  0  0  0  1 52]]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        54
           1       0.95      1.00      0.97        55
           2       1.00      1.00      1.00        53
           3       1.00      1.00      1.00        55
           4       0.98      1.00      0.99        54
           5       1.00      0.98      0.99        55
           6       0.98      1.00      0.99        54
           7       1.00      1.00      1.00        54
           8       0.98      0.92      0.95      

# Análise Crítica dos Resultados Digits:

A matriz de confusão com K = 3 e 31 apresentou as classes de dígitos (0 a 9) em sua maioria com classificações corretas, exibindo poucos erros. Na matriz, podemos observar que a classe 0, com K = 3, teve 54 previsões corretas, enquanto com K = 31, teve 53 acertos. A classe 1, em ambos K = 3 e 31, obteve 55 e 51 acertos, respectivamente. Os erros apresentados ocorreram na classe 3 com K = 3, que foi classificada erroneamente como 9, na classe 8, classificada erroneamente como 9, e, por fim, na classe 9, classificada incorretamente como 8. O segundo experimento, com K = 13, apresentou diversos erros, especialmente na classe 8, que teve apenas 44 previsões corretas, muito abaixo dos outros experimentos.

Porém, nossas taxas de precisão atingiram 98,7%, um bom índice de acerto, equiparando-se ao resultado obtido com o scikit-learn para K = 3, enquanto k = 13 com 97.59%%, porcetagem apresentou maior nivel de acerto que o experimento do sckit-learn com 97.41%. No entanto, nosso experimento com K = 31 a sua taxa de precisão foi prejudicada pelos erros encontrados, com 95.56%% de acertos. Portanto, essa métrica por si só pode não ser suficiente para entender completamente o desempenho do nosso modelo, especialmente se tivermos um caso de classes desbalanceadas.

No experimento com K = 3, a precisão varia de 0,95 para a classe 1 a 1,00 para as demais classes. Já para K = 31, a precisão varia de 0,92 para a classe 7 a 1,00 para as demais classes. A revocação também se mantém alta, variando de 0,92 para a classe 1 a 1,00 para as outras classes. Para K = 13 e 31, essa taxa varia bastante, entre 0,83 e 0,87 para algumas classes, chegando a 1,00 para outras. Por fim, o F1-score, que é a média harmônica entre precisão e revocação, é alto (0,95 a 1,00) para todas as classes, indicando um bom equilíbrio entre essas duas métricas, principalmente para K = 3. No entanto, para K = 13 e 31, essa métrica varia entre 0,92 e 1,00 e 0,87 e 1,00, respectivamente.

Para melhorar o desempenho do modelo, seria interessante investigar as falhas apresentadas ou mesmo aumentar a quantidade de dados, caso isso seja possível. Porém, como se trata de um dataset constituído de dados já consolidados, sugerimos um ajuste dos parâmetros para melhorar a performance do modelo.

# 2 - Segunda base de dados - Iris

In [ ]:
# carregamento da base de dados e verificação do formato
X_iris, y_iris = load_iris(return_X_y=True)
print(X_iris.shape)
print(y_iris.shape)

(150, 4)
(150,)


In [ ]:
# visualização dos dados em DataFrame
x = pd.DataFrame(X_iris)
y = pd.DataFrame(y_iris)
x.head()

,0,1,2,3
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [ ]:
# houldout/separação dos dados em treinamento e teste
X_train_iris, X_test_iris, y_train_iris, y_test_iris = train_test_split(X_iris, y_iris, test_size=0.3, random_state=42, stratify=y)

In [ ]:
# verificação dos formatos
print("X_test_iris: ", X_test_iris.shape)
print("y_test_iris: ", y_test_iris.shape)
print("X_train_iris: ", X_train_iris.shape)
print("y_train_iris: ", y_train_iris.shape)

X_test_iris:  (45, 4)
y_test_iris:  (45,)
X_train_iris:  (105, 4)
y_train_iris:  (105,)


# Chamada das funções - Iris

In [ ]:
# Inicia o timer
tempo_inicial1 = time.time()

# Chamada com K = 3, com KNN manual e Sckiti-Learn, Matriz de COnfusão e Classification Report
print(knn_manual_acuracia(3, X_train_iris, y_train_iris, X_test_iris, y_test_iris))
print(knn_scikit_learn(3, X_train_iris, y_train_iris, X_test_iris, y_test_iris))
predicted_manual_knn, accuracy_manual_knn = knn_manual_rotulo(3, X_train_iris, y_train_iris, X_test_iris, y_test_iris)
print(classification_report(y_test_iris, predicted_manual_knn))

# Para o timer e calcula o tempo de execução
tempo_final1 = time.time()
tempo_total1 = tempo_final1 - tempo_inicial1
print(f"Tempo de execução com k = 3: {tempo_total1:.2f} segundos\n")

# Inicia o timer
tempo_inicial1 = time.time()

# Chamada com K = 13, com KNN manual e Sckiti-Learn, Matriz de COnfusão e Classification Report
print(knn_manual_acuracia(13, X_train_iris, y_train_iris, X_test_iris, y_test_iris))
print(knn_scikit_learn(13, X_train_iris, y_train_iris, X_test_iris, y_test_iris))
predicted_manual_knn, accuracy_manual_knn = knn_manual_rotulo(13, X_train_iris, y_train_iris, X_test_iris, y_test_iris)
print(classification_report(y_test_iris, predicted_manual_knn))

# Para o timer e calcula o tempo de execução
tempo_final1 = time.time()
tempo_total1 = tempo_final1 - tempo_inicial1
print(f"Tempo de execução com k = 13: {tempo_total1:.2f} segundos\n")

# Inicia o timer
tempo_inicial1 = time.time()

# Chamada com K = 31, com KNN manual e Sckiti-Learn, Matriz de COnfusão e Classification Report
print(knn_manual_acuracia(31, X_train_iris, y_train_iris, X_test_iris, y_test_iris))
print(knn_scikit_learn(31, X_train_iris, y_train_iris, X_test_iris, y_test_iris))
predicted_manual_knn, accuracy_manual_knn = knn_manual_rotulo(31, X_train_iris, y_train_iris, X_test_iris, y_test_iris)
print(classification_report(y_test_iris, predicted_manual_knn))

# Para o timer e calcula o tempo de execução
tempo_final1 = time.time()
tempo_total1 = tempo_final1 - tempo_inicial1
print(f"Tempo de execução com k = 31: {tempo_total1:.2f} segundos\n")

Acurácia do nosso experimento com K = 3: 95.56% ou 0.95556
Acurácia Scikit-Learn com K = 3: 95.56% ou 0.95556 
 Matrix de Confusão:
[[15  0  0]
 [ 0 15  0]
 [ 0  2 13]]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       0.88      1.00      0.94        15
           2       1.00      0.87      0.93        15

    accuracy                           0.96        45
   macro avg       0.96      0.96      0.96        45
weighted avg       0.96      0.96      0.96        45

Tempo de execução com k = 3: 0.06 segundos

Acurácia do nosso experimento com K = 13: 93.33% ou 0.93333
Acurácia Scikit-Learn com K = 13: 93.33% ou 0.93333 
 Matrix de Confusão:
[[15  0  0]
 [ 0 14  1]
 [ 0  2 13]]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       0.88      0.93      0.90        15
           2       0.93      0.87      0.90        15

    accuracy   

# Análise Crítica dos Resultados Iris:

Utilizamos neste experimento três valores de K diferentes: 3, 13 e 31. Podemos observar que houve resultados semelhantes na acurácia entre nossos experimentos com o dataset Iris e os obtidos pelo scikit-learn, com as seguintes porcentagens: 95,56%, 93,33% e 93,33%. Percebemos que essa paridade de resultados se deve à pequena quantidade de dados presentes neste dataset.

A matriz de confusão divide as classes em três tipos (0, 1, 2), onde observamos um acerto completo na classe 0, sem nenhum erro para ambos os casos com K = 3, 13 e 31. A classe 1 apresentou 2 erros de classificação, sendo confundida com a classe 2 para K = 13 e 31, enquanto a classe 2 apresentou 2 erros, sendo classificada incorretamente como pertencente à classe 1.

A acurácia do modelo com K = 3 foi de 95,56%, o valor mais elevado em comparação aos demais experimentos com K = 13 e 31. No entanto, sozinha, a acurácia não proporciona um entendimento profundo sobre o desempenho do modelo, especialmente se houver a possibilidade de dados desbalanceados.

A precisão nas classes 0 e 2 foi excelente, com um valor de 1,00 para K = 3, e para K = 13 e 31, variou de 0,93 a 1,00, indicando que todas as previsões para essas classes foram corretas, ou quase corretas. Em contrapartida, a classe 1 apresentou uma precisão de 0,88 para todos os valores de K, revelando alguns erros de classificação.

A revocação para K = 3, 13 e 31 das classes 0 e 2 foi alta, ambas variando de 1,00 a 0,93, o que significa que quase todos os exemplos dessas classes foram identificados pelo modelo com um certo nível de acerto. No entanto, a revocação da classe 2 foi um pouco menor, com um valor de 0,87 para todos os valores de K, refletindo os erros observados na matriz de confusão.

O F1-score, que é a média harmônica entre precisão e revocação, variou entre 0,90 e 1,00 para todas as classes e valores de K, indicando um bom equilíbrio entre essas duas métricas.

Assim como no exemplo do Digits, sugerimos a investigação dos erros apresentados na matriz de confusão e a possibilidade de utilizar técnicas de ajuste fino no modelo para melhorar seu desempenho, devido à pouca quantidade de dados encontrados neste dataset Iris.